In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import json

In [2]:
#Define Executable Path for ChromeDriver to use for web scraping

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Define base endpoint URL. Other url's will be built off this endpoint.

espn_url = 'https://www.espn.com'

In [4]:
# Define UNC extension and URL

tarheel_ext = '/mens-college-basketball/team/_/id/153/north-carolina-tar-heels'
tarheel_team_page = espn_url + tarheel_ext

browser.visit(tarheel_team_page)

In [5]:
#Use Beautiful Soup HTML.parser to extract relvant details about news articles

html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# Extract list of games

game_listings = soup.find_all('section', class_="club-schedule")[0].find_all('li')
game_listings

[<li><a class="" href="/mens-college-basketball/game?gameId=401310911" name="&amp;lpos=mens-college-basketball:teamclubhouse:schedule:post" rel=""><div class="logo"><img class="imageLoaded" data-behavior="fix_broken_images" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/ncaa/500/275.png&amp;h=40&amp;w=40"/></div><div class="game-info">vs <span class="rank">9</span> WISC</div><div class="game-meta"><div class="game-result loss">L</div><div class="score">85-62</div></div></a></li>,
 <li><a class="" href="/mens-college-basketball/game?gameId=401252794" name="&amp;lpos=mens-college-basketball:teamclubhouse:schedule:regular" rel=""><div class="logo"><img class="imageLoaded" data-behavior="fix_broken_images" src="https://a.espncdn.com/combiner/i?img=/i/teamlogos/ncaa/500/232.png&amp;h=40&amp;w=40"/></div><div class="game-info">vs  COFC</div><div class="game-meta"><div class="game-result win">W</div><div class="score">79-60</div></div></a></li>,
 <li><a class="" href="/mens-college-ba

In [7]:
game_urls = []
for game in game_listings:
    new_url = game.a['href']
    game_url = espn_url + new_url
    browser.visit(game_url)
    game_urls.append(game_url)

In [8]:
game_urls

['https://www.espn.com/mens-college-basketball/game?gameId=401310911',
 'https://www.espn.com/mens-college-basketball/game?gameId=401252794',
 'https://www.espn.com/mens-college-basketball/game?gameId=401255064',
 'https://www.espn.com/mens-college-basketball/game?gameId=401256578',
 'https://www.espn.com/mens-college-basketball/game?gameId=401256579',
 'https://www.espn.com/mens-college-basketball/game?gameId=401256095',
 'https://www.espn.com/mens-college-basketball/game?gameId=401268610',
 'https://www.espn.com/mens-college-basketball/game?gameId=401270409',
 'https://www.espn.com/mens-college-basketball/game?gameId=401260181',
 'https://www.espn.com/mens-college-basketball/game?gameId=401265053',
 'https://www.espn.com/mens-college-basketball/game?gameId=401274413',
 'https://www.espn.com/mens-college-basketball/game?gameId=401265057',
 'https://www.espn.com/mens-college-basketball/game?gameId=401260205',
 'https://www.espn.com/mens-college-basketball/game?gameId=401276235',
 'http

In [10]:
game_jsons = []
for game in game_urls:
    try:
        browser.visit(game)
        temp_html = browser.html
        temp_soup = bs(temp_html, 'html.parser')
        boxxy_score_endpoint = temp_soup.find('li', class_="sub boxscore").a['href']
        another_temp = espn_url + boxxy_score_endpoint
        browser.visit(another_temp)
        another_html = browser.html
        another_soup = bs(another_html, 'html.parser')
        teams = another_soup.find('div',class_="row-wrapper")
        away_team = teams.find('div',class_="col column-one gamepackage-away-wrap").find('div',class_='team-name').text
        home_team = teams.find('div',class_="col column-two gamepackage-home-wrap").find('div',class_='team-name').text
        print(f'The Away Team is {away_team}')
        print(f'The Home Team is {home_team}')
        tables = pd.read_html(another_temp)
        heels = tables[2]
        heels.drop(heels.tail(2).index,inplace=True)
        players = heels['Starters']['Bench']
        minutes = heels['MIN']['MIN']
        field_goals = heels['FG']['FG']
        free_throws = heels['FT']['FT']
        three_pointers = heels['3PT']['3PT']
        off_reb = heels['OREB']['OREB']
        def_reb = heels['DREB']['DREB']
        total_reb = heels['REB']['REB']
        assists = heels['AST']['AST']
        steals = heels['STL']['STL']
        blocks = heels['BLK']['BLK']
        turnovers = heels['TO']['TO']
        points = heels['PTS']['PTS']
        players_names = []
        for player in players:
            try:
                #print(player)
                length = len(player)
                splitter = (length-1)/2
                splitter_int = int(splitter)
                player_name = player[:splitter_int]
                players_names.append(player_name)
            except TypeError:
                print('Doesnt matter!')
        game_df = pd.DataFrame({
        "Player": players_names,
        "MP": minutes,
        "FG": field_goals,
        "FT": free_throws,
        "3P": three_pointers,
        "OREB": off_reb,
        "DREB": def_reb,
        "TOT REB": total_reb,
        "AST": assists,
        "STL": steals,
        "BLK": blocks,
        "TO": turnovers,
        "PTS": points
        })
        game_df['MP'] = game_df['MP'].astype(int)
        game_df['FGA'] = game_df['FG'].str.split(pat="-")[0][0]
        game_df['FGM'] = game_df['FG'].str.split(pat="-")[0][1]
        game_df['FTA'] = game_df['FT'].str.split(pat="-")[0][0]
        game_df['FTM'] = game_df['FT'].str.split(pat="-")[0][1]
        game_df['3PA'] = game_df['3P'].str.split(pat="-")[0][0]
        game_df['3PM'] = game_df['3P'].str.split(pat="-")[0][1]
        game_df['OREB'] = game_df['OREB'].astype(int)
        game_df['DREB'] = game_df['DREB'].astype(int)
        game_df['TOT REB'] = game_df['TOT REB'].astype(int)
        game_df['AST'] = game_df['AST'].astype(int)
        game_df['STL'] = game_df['STL'].astype(int)
        game_df['BLK'] = game_df['BLK'].astype(int)
        game_df['TO'] = game_df['TO'].astype(int)
        game_df['PTS'] = game_df['PTS'].astype(int)
        cleaned_game_df = game_df.drop(['FG','FT','3P'],axis=1)
        cleaned_game_df = cleaned_game_df[['Player','MP','FGA','FGM','3PA','3PM','FTA','FTM','OREB','DREB','TOT REB','AST','STL', \
                                      'BLK', 'TO', 'PTS']]
        indiv_game_json_frame = {'ID':0,
                             'Home_Away': 'Away',
                             'Game_Info': cleaned_game_df.to_json()
                            }
        game_jsons.append(indiv_game_json_frame)
    except AttributeError:
        print('Game was cancelled or postponed')

The Away Team is Wisconsin
The Home Team is North Carolina
The Away Team is Charleston
The Home Team is North Carolina
The Away Team is North Carolina
The Home Team is UNLV
The Away Team is North Carolina
The Home Team is Stanford
The Away Team is Texas
The Home Team is North Carolina
The Away Team is North Carolina
The Home Team is Iowa
The Away Team is North Carolina Central
The Home Team is North Carolina
The Away Team is North Carolina
The Home Team is Kentucky
The Away Team is North Carolina
The Home Team is NC State
The Away Team is North Carolina
The Home Team is Georgia Tech
The Away Team is Notre Dame
The Home Team is North Carolina
The Away Team is North Carolina
The Home Team is Miami
Game was cancelled or postponed
The Away Team is Syracuse
The Home Team is North Carolina
The Away Team is North Carolina
The Home Team is Florida State
The Away Team is Wake Forest
The Home Team is North Carolina
The Away Team is NC State
The Home Team is North Carolina
The Away Team is North 

In [32]:
#with open("sample.json", "w") as outfile: 
   #json.dump(indiv_game_json_frame, outfile)

In [ ]:
home_team_2 = teams.find('div',class_="col column-two gamepackage-home-wrap").find_all('td', class_="name")
players_full_names = []
for player in home_team_2:
    try:
        player_url = player.a['href']
        browser.visit(player_url)
        player_html = browser.html
        player_soup = bs(player_html, 'html.parser')
        player_info = player_soup.find('div', class_='PlayerHeader__Main flex items-center')
        player_image = player_info.find('div',class_='Image__Wrapper Image__Wrapper--relative').img['src']
        player_name_bin = player_info.find('h1',class_='PlayerHeader__Name flex flex-column ttu fw-bold pr4 h3').find_all('span')
        first_name = player_name_bin[0].text
        last_name = player_name_bin[1].text
        players_full_names.append(f'{first_name} {last_name}')
    except TypeError:
        print('End of Dataset')
        
players_full_names

In [ ]:
s = tables[2]['FG']['FG']
splits = s.str.split(pat="-")
splits

In [ ]:
for data in splits:
    try:
        FGM = data[0]
        FGA = data[1]
        print(f'{FGM} shots were made out of {FGA} taken')
    except IndexError:
        print('N/A')